### MY data processing

In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path1 = r'C:\Users\Jerry Wong\OneDrive - BBMSL Limited\Documents - BBMSL Alliance Team\02.0 Data Analysis\02.01 Raw Data\Regional\MY\Malayasia transactions\Malayasia Transaction.csv'
df1 = pd.read_csv(file_path1)

In [3]:
df1['Date'] = pd.to_datetime(df1['Date'], dayfirst=True)
df1['Month'] = df1['Date'].dt.to_period('M')

#### remove specific rows

In [4]:
df1 = df1[~df1['Status'].str.contains('cancelled|failed', na=False)]

#### fill in missing values

In [20]:
# df1.loc[(df1['Channel'] == 'DuitNow QR Offline') & (df1['Card Brand'].isna()), 'Card Brand'] = 'DUITNOW'
# df1.loc[((df1['Channel'] == 'AlipayPlus Offline') | (df1['Channel'] == 'Alipay-Spot')) & (df1['Card Brand'].isna()), 'Card Brand'] = 'ALIPAY'
# df1.loc[(df1['Channel'] == 'Mastercard') & (df1['Card Brand'].isna()), 'Card Brand'] = 'MASTERCARD'
# df1.loc[(df1['Channel'] == 'Visa') & (df1['Card Brand'].isna()), 'Card Brand'] = 'VISA'

In [7]:
df1['Channel'] = np.select([df1['Channel'].str.lower().str.contains('duitnow', na=False),
                            df1['Channel'].str.lower().str.contains('alipay', na=False),
                            df1['Channel'].str.lower().str.contains('debit', na=False)],
                           ['DUITNOW',
                            'ALIPAY',
                            df1['Card Brand']],
                           df1['Channel'].str.upper())

In [9]:
df1 = df1.groupby(['Month', 'Merchant / Sub Merchant ID', 'Channel', 'Card Type', 'Card Country'], as_index=False, dropna=False).agg({'Bill Amt': 'sum', 'Actual Amt': 'sum', 'Transaction ID': 'count'})

In [12]:
df1.rename(columns={'Transaction ID': 'No of Transaction'}, inplace=True)

In [10]:
df1['Channel'].value_counts()

Channel
VISA          572
MASTERCARD    383
DUITNOW        63
ALIPAY          4
UNIONPAY        1
Name: count, dtype: int64

In [13]:
df1.to_csv('MYdata.csv', index=False)

#### Transform Merchant ID

In [14]:
df1['Merchant ID'] = np.where(df1['Merchant / Sub Merchant ID'].str.islower(), df1['Merchant / Sub Merchant ID'].str[:-2], np.where(df1['Merchant / Sub Merchant ID'].str.startswith('BX', na=False), df1['Merchant / Sub Merchant ID'].str[2:], df1['Merchant / Sub Merchant ID']))

### MY database processing

#### Read the Excel file, Read the Sheet 'Razer', and set the second row as the header

In [15]:
database_path = 'Razer Submission Tracker  - Copy.xlsx'
df2 = pd.read_excel(database_path, sheet_name='Razer', header=1)

#### Transform Merchant ID

In [16]:
df2['Merchant ID'] = np.where(df2['Merchant Admin ID'].str.lower().str.startswith('bx', na=False), df2['Merchant Admin ID'].str[2:], df2['Merchant Admin ID'])

#### Group the dataframe

In [17]:
df_filtered = df2[['Company Name', 'Hardware', 'Merchant Admin ID', 'Sub Merchant ID', 'Submission Date', 'Rejected Date', 'Approved Date', 'Sales', 'Merchant ID']]

#### Left merge excel file

In [18]:
df_filtered = df_filtered.merge(df1, on='Merchant ID', how='left').drop(columns='Merchant ID')

In [19]:
df_filtered

,Company Name,Hardware,Merchant Admin ID,Sub Merchant ID,Submission Date,Rejected Date,Approved Date,Sales,Month,Merchant / Sub Merchant ID,Channel,Card Type,Card Country,Bill Amt,Actual Amt,No of Transaction
0,GALAXY PET LAND,NaN,NaN,NaN,2023-12-07,2024-06-02,NaT,Clement,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MY MOTOR SERVICE,Fiuu Terminal,Bxmymotorservic,mymotorservic01,2024-01-04,NaT,2024-01-29,Jason,2024-03,mymotorservic01,VISA,CREDIT,MALAYSIA,170.0,167.96,1.0
2,MY MOTOR SERVICE,Fiuu Terminal,Bxmymotorservic,mymotorservic01,2024-01-04,NaT,2024-01-29,Jason,2024-03,mymotorservic01,VISA,DEBIT,MALAYSIA,411.2,406.27,2.0
3,MY MOTOR SERVICE,Fiuu Terminal,Bxmymotorservic,mymotorservic01,2024-01-04,NaT,2024-01-29,Jason,2024-04,mymotorservic01,MASTERCARD,CREDIT,MALAYSIA,20.2,19.96,1.0
4,MY MOTOR SERVICE,Fiuu Terminal,Bxmymotorservic,mymotorservic01,2024-01-04,NaT,2024-01-29,Jason,2024-05,mymotorservic01,MASTERCARD,CREDIT,MALAYSIA,45.0,44.46,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,FIRST GRADE RESOURCES SDN BHD\t,Fiuu Terminal,BXfirstgraderes,firstgraderes01,2024-05-28,NaT,2024-05-30,A'fifah,2024-06,firstgraderes01,VISA,CREDIT,MALAYSIA,68319.9,67636.70,2.0
1044,YOU MING PORTAL SDN BHD,POS,NaN,NaN,2024-06-19,2024-06-21,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1045,MUZHI AUTO ENTERPRISE,Fiuu Terminal,BXmuzhiautoente,muzhiautoente01,2024-06-25,NaT,2024-06-26,Clement,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1046,DOKET DAF ENTERPRISE,Fiuu Terminal,BXdoketdafenter,doketdafent01,2024-07-03,NaT,2024-07-09,Clement,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_filtered.to_csv('MYleftJoin.csv', index=False)

### Pivot

In [41]:
pivot_df = df1.astype({'Month': str}).groupby(['Month', 'Merchant ID', 'Channel'], as_index=False)[['Actual Amt', 'No of Transaction']].sum()
pivot_df = pivot_df.pivot_table(
    values=['Actual Amt', 'No of Transaction'],
    index=['Month', 'Merchant ID'],
    columns=['Channel'],
    fill_value=0
)

In [42]:
pivot_df[('Actual Amt', 'Total')] = pivot_df['Actual Amt'].sum(axis=1)
pivot_df[('No of Transaction', 'Total')] = pivot_df['No of Transaction'].sum(axis=1)

In [43]:
pivot_df = pivot_df.pivot_table(
    values=['Actual Amt', 'No of Transaction'],
    index=['Merchant ID'],
    columns=['Month'],
    fill_value=0
)

In [44]:
pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]

In [45]:
pivot_df

,Actual Amt_ALIPAY_2024-02,Actual Amt_ALIPAY_2024-03,Actual Amt_ALIPAY_2024-04,Actual Amt_ALIPAY_2024-05,Actual Amt_ALIPAY_2024-06,Actual Amt_ALIPAY_2024-07,Actual Amt_DUITNOW_2024-02,Actual Amt_DUITNOW_2024-03,Actual Amt_DUITNOW_2024-04,Actual Amt_DUITNOW_2024-05,...,No of Transaction_UNIONPAY_2024-04,No of Transaction_UNIONPAY_2024-05,No of Transaction_UNIONPAY_2024-06,No of Transaction_UNIONPAY_2024-07,No of Transaction_VISA_2024-02,No of Transaction_VISA_2024-03,No of Transaction_VISA_2024-04,No of Transaction_VISA_2024-05,No of Transaction_VISA_2024-06,No of Transaction_VISA_2024-07
Merchant ID,,,,,,,,,,,,,,,,,,,,,
20saifoodstal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,43.0,8.0
MRAboxsmy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aisynabawient,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,21.0,29.0,9.0,41.0,44.0,35.0
akautomotivee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
alphabiomedic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.38,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
siangmalamkop,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,4.0,24.0,21.0,16.0,15.0,0.0
themelvinsres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10707.05,1430.84,0.0,...,0.0,0.0,0.0,0.0,0.0,644.0,148.0,0.0,0.0,0.0
tumezkitchen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,48.0


In [46]:
database_path = 'Razer Submission Tracker  - Copy.xlsx'
df = pd.read_excel(database_path, sheet_name='Razer', header=1)
df['Merchant ID'] = np.where(df2['Merchant Admin ID'].str.lower().str.startswith('bx', na=False), df['Merchant Admin ID'].str[2:], df['Merchant Admin ID'])
df = df[['Company Name', 'Hardware', 'Merchant Admin ID', 'Sub Merchant ID', 'Submission Date', 'Rejected Date', 'Approved Date', 'Sales', 'Merchant ID']]
df = df.merge(pivot_df, on='Merchant ID', how='left').drop(columns='Merchant ID')

In [47]:
df

,Company Name,Hardware,Merchant Admin ID,Sub Merchant ID,Submission Date,Rejected Date,Approved Date,Sales,Actual Amt_ALIPAY_2024-02,Actual Amt_ALIPAY_2024-03,...,No of Transaction_UNIONPAY_2024-04,No of Transaction_UNIONPAY_2024-05,No of Transaction_UNIONPAY_2024-06,No of Transaction_UNIONPAY_2024-07,No of Transaction_VISA_2024-02,No of Transaction_VISA_2024-03,No of Transaction_VISA_2024-04,No of Transaction_VISA_2024-05,No of Transaction_VISA_2024-06,No of Transaction_VISA_2024-07
0,GALAXY PET LAND,NaN,NaN,NaN,2023-12-07,2024-06-02,NaT,Clement,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MY MOTOR SERVICE,Fiuu Terminal,Bxmymotorservic,mymotorservic01,2024-01-04,NaT,2024-01-29,Jason,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,1.0
2,RZA N 9 ENTERPRISE,Fiuu Terminal,BXrzan9enterpri,rzan9enterpri01,2024-01-05,NaT,2024-01-29,A'fifah,0.0,0.0,...,0.0,0.0,0.0,0.0,44.0,38.0,19.0,92.0,72.0,28.0
3,AISY NABAWI ENTERPRISE,Fiuu Terminal,Bxaisynabawient,aisynabawient01,2024-01-05,NaT,2024-02-02,Ulul,0.0,0.0,...,0.0,0.0,0.0,0.0,21.0,29.0,9.0,41.0,44.0,35.0
4,THE MELVINS REOURCES,Fiuu Terminal,Bxthemelvinsres,themelvinsres01,2024-01-05,NaT,2024-01-29,A'fifah,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,644.0,148.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,FIRST GRADE RESOURCES SDN BHD\t,Fiuu Terminal,BXfirstgraderes,firstgraderes01,2024-05-28,NaT,2024-05-30,A'fifah,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
82,YOU MING PORTAL SDN BHD,POS,NaN,NaN,2024-06-19,2024-06-21,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,MUZHI AUTO ENTERPRISE,Fiuu Terminal,BXmuzhiautoente,muzhiautoente01,2024-06-25,NaT,2024-06-26,Clement,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,DOKET DAF ENTERPRISE,Fiuu Terminal,BXdoketdafenter,doketdafent01,2024-07-03,NaT,2024-07-09,Clement,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df.to_csv('MY.csv', index=False)